## Christian Campbell - Milestone 5

In [47]:
import sqlite3 as sql
import numpy as np
import requests
import re
import ssl
from bs4 import BeautifulSoup
import urllib.request, urllib.parse
from urllib.error import HTTPError, URLError
import socket
import json
import matplotlib.pyplot as plt
import ast
import seaborn as sns
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp

## Importing website data

In [2]:
database = "website_crime_data"
connection = sql.connect(database)

In [3]:
# The code below checks if the connection open.
def is_opened(connection):
    try:
        connection.execute("SELECT * FROM website_crime_data LIMIT 1")
        return True
    except sql.ProgrammingError as e:
        print("Connection closed {}".format(e))
        return False

In [4]:
print(is_opened(connection))

True


In [7]:
#This code brings up the table
website_data="website_crime_data"
query = f"SELECT * FROM {website_data}"
try:
    web_data = pd.read_sql(query, connection)
    print(web_data)
except sql.Error as e:
    print(f"Error reading data from the table: {e}")

                 Location Violent crime rate Homicide rate Rape rate  \
0                 Alabama              409.1          10.9      29.6   
1                  Alaska              758.9           9.5     134.0   
2                 Arizona              431.5           6.8      44.1   
3                Arkansas              645.3          10.2      76.0   
4              California              499.5           5.7      37.4   
5                Colorado              492.5           6.4      63.4   
6             Connecticut              150.0           3.8      18.1   
7                Delaware              383.5           4.8      22.0   
8    District of Columbia              812.3          29.3      41.5   
9                 Florida              258.9           5.0      30.2   
10                Georgia              367.0           8.2      36.4   
11                 Hawaii              259.6           2.1      37.9   
12                  Idaho              241.4           2.7      

In [43]:
# This code plots a pie chart for Violent crime rate in the US States.
# The chart is interactive. 
fig = px.pie(web_data, names='Location', values='Violent crime rate', title='Violent Crime Rate (2022)')
fig.show()

## Importing flatfile data

In [9]:
# This code is to establish a connection to the flatfile database
database2 = "flatfile_crime_data"
conn = sql.connect(database2)

In [12]:
# The code below checks if the connection open.
def is_opened(conn):
    try:
        conn.execute("SELECT * FROM flatfile_crime_data LIMIT 1")
        return True
    except sql.ProgrammingError as e:
        print("Connection closed {}".format(e))
        return False

In [13]:
print(is_opened(conn))

True


In [50]:
#This code brings up the flatfile table
flat_data="flatfile_crime_data"
query = f"SELECT * FROM {flat_data}"
try:
    flat_data = pd.read_sql(query, conn)
    print(flat_data)
except sql.Error as e:
    print(f"Error reading data from the table: {e}")

      jurisdiction  includes_jails  year  prisoner_count  \
0          ALABAMA               0  2001           24741   
1           ALASKA               1  2001            4570   
2          ARIZONA               0  2001           27710   
3         ARKANSAS               0  2001           11489   
4       CALIFORNIA               0  2001          157142   
..             ...             ...   ...             ...   
795       VIRGINIA               0  2016           29882   
796     WASHINGTON               0  2016           17228   
797  WEST VIRGINIA               0  2016            5899   
798      WISCONSIN               0  2016           23163   
799        WYOMING               0  2016            2352   

     crime_reporting_change  crimes_estimated  state_population  \
0                       0.0               0.0         4468912.0   
1                       0.0               0.0          633630.0   
2                       0.0               0.0         5306966.0   
3          

In [16]:
#This code plots a bar graph for the US prisoner population for 2016
filtered_data = flat_data[flat_data['year'] == 2016]
fig = px.bar(filtered_data, x='jurisdiction', y='prisoner_count', title='Prisoner Count by Jurisdiction in 2016')
fig.show()

## Importing api data

In [32]:
# This code is to establish a connection to the api database
db3 = "api_crime_data"
conn2 = sql.connect(db3)

In [33]:
# The code below checks if the connection open.
def is_opened(conn2):
    try:
        conn2.execute("SELECT * FROM api_crime_data LIMIT 1")
        return True
    except sql.ProgrammingError as e:
        print("Connection closed {}".format(e))
        return False

In [34]:
print(is_opened(conn2))

True


In [45]:
#This code brings up the api table
api_data="api_crime_data"
query = f"SELECT * FROM {api_data}"
try:
    api_data = pd.read_sql(query, conn2)
    print(api_data)
except sql.Error as e:
    print(f"Error reading data from the table: {e}")

             State  Year  Aggravated Assault  Manslaughter by Negligence  \
0          Alabama  2016              4887.0                        13.0   
1           Alaska  2016              1845.0                         5.0   
2          Arizona  2016              9402.0                        42.0   
3         Arkansas  2016              3929.0                        10.0   
4       California  2016             87210.0                       224.0   
..             ...   ...                 ...                         ...   
294       Virginia  2021              4957.0                        55.0   
295     Washington  2021              6058.0                        16.0   
296  West Virginia  2021              1246.0                         4.0   
297      Wisconsin  2021              5394.0                        53.0   
298        Wyoming  2021               279.0                         2.0   

     Murder and Nonnegligent Manslaughter    Rape  Robbery  Simple Assault  \
0        

In [46]:
# This code chunk plots the line chart for the following five states. 
#The chart is interactive.
selected_states = ['New York', 'California', 'Texas', 'Georgia', 'Florida']
filtered_data = api_data[api_data['State'].isin(selected_states)]
fig = px.line(filtered_data, x='Year', y='Rape', color='State', markers=True,
              title='Rape Between 2016 and 2021')
fig.show()

## First Visualization across two sources

In [55]:
# In order to create this visual, I took values from the flat_data table and the api_data table
# Filter flat_data for CALIFORNIA and 2016
filtered_flat_data = flat_data[(flat_data['jurisdiction'] == 'CALIFORNIA') & (flat_data['year'] == 2016)]

# Filter api_data for California and 2016
filtered_api_data = api_data[(api_data['State'] == 'California') & (api_data['Year'] == 2016)]

# Create subplots with shared y-axis
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['Aggravated Assault', 'Simple Assault'],
                       shared_yaxes=True)

# Add bar chart for flat_data
fig.add_trace(
    go.Bar(x=filtered_flat_data['jurisdiction'], y=filtered_flat_data['agg_assault'], name='Agg Assault'),
    row=1, col=1
)

# Add bar chart for api_data
fig.add_trace(
    go.Bar(x=filtered_api_data['State'], y=filtered_api_data['Simple Assault'], name='Simple Assault'),
    row=1, col=2
)

# Update layout
fig.update_layout(title_text='Combined Bar Charts for Aggravated Assault and Simple Assault in California (2016)',
                  showlegend=False)

# Show the plot
fig.show()

## Second Visualization across two sources

In [62]:
# In order to create this visual, I took values from the flat_data table and api_data table.
# The graph is interactive.
filtered_flat = flat_data[(flat_data['jurisdiction'].isin(['CALIFORNIA', 'TEXAS', 'GEORGIA'])) & (flat_data['year'].between(2001, 2015))]

# Filter api_data for California, Texas, Florida
filtered_api = api_data[(api_data['State'].isin(['California', 'Texas', 'Georgia']))]

# Create a line graph
fig = go.Figure()

# Add line for flat_data
for jurisdiction in ['CALIFORNIA', 'TEXAS', 'GEORGIA']:
    fig.add_trace(
        go.Scatter(x=filtered_flat[filtered_flat['jurisdiction'] == jurisdiction]['year'],
                   y=filtered_flat[filtered_flat['jurisdiction'] == jurisdiction]['robbery'],
                   mode='lines+markers',
                   name=f'{jurisdiction} - flat_data')
    )

# Add line for api_data
for state in ['California', 'Texas', 'Georgia']:
    fig.add_trace(
        go.Scatter(x=filtered_api[filtered_api['State'] == state]['Year'],
                   y=filtered_api[filtered_api['State'] == state]['Robbery'],
                   mode='lines+markers',
                   name=f'{state} - api_data')
    )

# Update layout
fig.update_layout(title_text='Robbery from 2001 to 2021',
                  xaxis_title='Year',
                  yaxis_title='Robbery Count',
                  legend=dict(x=0, y=1, traceorder='normal'))

# Show the plot
fig.show()

## Merging all three tables

In [68]:
# I couldn't get this to work. 
# Merge flat_data and api_data on 'jurisdiction' and 'State'
merged_data1 = pd.merge(flat_data, api_data, left_on='jurisdiction', right_on='State', how='outer')

# Merge the result with website_data on 'Location'
final_merged_data = pd.merge(merged_data1, website_data, left_on='jurisdiction', right_on='Location', how='outer')

# Drop redundant columns if needed
final_merged_data = final_merged_data.drop(['State'], axis=1)

# Display the final merged data
print(final_merged_data)

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed